In [1]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe, get_extreme_values, create_directory_structure, save_table, save_report, pd, assign_quartile, rank_key_size, calculate_rank
# settings
region_column_name = 'Region'

category = 'security_layer'
column_name_to_results_global = 'Global #'
create_directory_structure()



source_df = load_data('security_layer_checker')

In [2]:
source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          458 non-null    int64 
 1   region                      458 non-null    object
 2   name                        458 non-null    object
 3   category                    458 non-null    object
 4   url                         458 non-null    object
 5   grade                       458 non-null    object
 6   SSLv2.0                     458 non-null    bool  
 7   SSLv3.0                     458 non-null    bool  
 8   TLSv1.0                     458 non-null    bool  
 9   TLSv1.1                     458 non-null    bool  
 10  TLSv1.2                     458 non-null    bool  
 11  TLSv1.3                     458 non-null    bool  
 12  dns_caa                     458 non-null    bool  
 13  issuer                      454 non-null    object

In [3]:
# sanity dataset
source_df.loc[source_df['grade'].isna(), 'grade'] = 'M'

In [4]:
# Analyze of CA (pub/Pvt)

# settings
sort_ascending = False
config = [
    {'table_name': 'ca_public', 'hei_type': 'Public'},
    {'table_name': 'ca_private', 'hei_type': 'Private'}
]
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']

    filtered_df = source_df.query(f'grade != "M" & category == "{hei_type}"').groupby('issuer').count()['url'].sort_values(ascending=False)
    top_5_df = filtered_df.head(5).reset_index()
    other_total = filtered_df[5:].sum()

    top_5_df.loc[5] = ['Others', other_total]
    top_5_df['percentual'] = (top_5_df['url'] / top_5_df['url'].sum()) * 100
    top_5_df.columns = ['Certificate Authority', 'Total #', 'Total %']
    top_5_df.reset_index(drop=True, inplace=True)

    # save to csv
    save_table(top_5_df, category=category, table_name=table_name)

In [5]:
# Analyze of key length by region

# settings
column_to_sort = 'Without SSL (Public) %'
sort_ascending = False
table_name = 'key_length_by_region'

columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without SSL by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Public)', criteria=f'grade == "M" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Private)', criteria=f'grade == "M" & {only_private}', columns_to_display=columns_to_display)
# Creating column with the distribution of Key Length by region
key_lengths = [256, 384, 1024, 2048, 3072, 4096]
for key_length in key_lengths:
    algorithm = 'RSA' if key_length >= 1024 else 'ECC'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{key_length} ({algorithm}) (Public)', criteria=f'grade != "M" & key_size == {key_length} & {only_public}', columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{key_length} ({algorithm}) (Private)', criteria=f'grade != "M" & key_size == {key_length} & {only_private}', columns_to_display=columns_to_display)


# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

ValueError: Percent sum is not equal to 100 for row 9: 98.50746268656717

In [ ]:
pub = source_df[source_df['category'] == 'Public']
criteria = 'key_size != 256 & key_size != 384 & key_size != 1024 & key_size != 2048 & key_size != 3072 & key_size != 4096'
strange = pub.query(criteria)
display(strange[['url', 'key_size', 'grade', 'issuer']])
# reescreva otimizadamente a consulta anterior


In [ ]:
key_lengths = source_df['key_size'].unique()
# Remova o valor zero, caso esteja presente
key_lengths = [key_length for key_length in key_lengths if key_length != 0]

# Ordenar os tamanhos de chave com base no mapeamento personalizado
key_lengths = sorted(key_lengths, key=rank_key_size)
print(key_lengths)

In [6]:
# Analyze of key length by region (Pub/Pvt)

# settings
column_to_sort = 'Without SSL %'
sort_ascending = False
config = [
    {'table_name': 'key_length_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'key_length_by_region_private', 'hei_type': 'Private'}
]
dfs = []
key_lengths = source_df['key_size'].unique()
key_lengths = [key_length for key_length in key_lengths if key_length != 0]
key_lengths = sorted(key_lengths, key=rank_key_size)

for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)

    # create columns
    # Column creation with distribution of records without SSL by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL', criteria=f'grade == "M" & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Creating column with the distribution of Key Length by region
    #deveria obter todos os valores desta coluna de forma dinamica
    for key_length in key_lengths:
        algorithm = 'RSA' if key_length >= 1024 else 'ECC'
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{key_length} ({algorithm})', criteria=f'grade != "M" & key_size == {key_length} & category == "{hei_type}"', columns_to_display=columns_to_display)

    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)

df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
percent_columns_pub = [col for col in df_combined.columns if col.endswith('%(pub)')]

# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')


ranks_columns = []
# Specify the columns to use
columns_to_use = ['8192 (RSA)', '384 (ECC)', '4096 (RSA)', '256 (ECC)']
# Apply the function to calculate the rank
for i, column in enumerate(columns_to_use):
    rank_colum = f'Rank {i}'
    ranks_columns.append(rank_colum)
    df_combined[rank_colum] = df_combined.apply(lambda row: calculate_rank(row, column), axis=1)

#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=ranks_columns, ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=ranks_columns, inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]


save_table(df_combined, category=category, table_name='key_length_by_region_combined')

,Region,Global #,Without SSL #,Without SSL %,2048 (RSA) #,2048 (RSA) %,3072 (RSA) #,3072 (RSA) %,256 (ECC) #,256 (ECC) %,4096 (RSA) #,4096 (RSA) %,384 (ECC) #,384 (ECC) %,8192 (RSA) #,8192 (RSA) %
0,Mecklenburg-Vorpommern,7,1,14.285714,0,0.000000,0,0.000000,0,0.000000,6,85.714286,0,0.000000,0,0.000000
1,Niedersachsen,21,1,4.761905,11,52.380952,0,0.000000,0,0.000000,9,42.857143,0,0.000000,0,0.000000
2,Baden-Württemberg,48,2,4.166667,22,45.833333,0,0.000000,5,10.416667,18,37.500000,1,2.083333,0,0.000000
3,Bayern,34,0,0.000000,15,44.117647,0,0.000000,2,5.882353,17,50.000000,0,0.000000,0,0.000000
4,Berlin,12,0,0.000000,5,41.666667,0,0.000000,2,16.666667,5,41.666667,0,0.000000,0,0.000000
5,Brandenburg,10,0,0.000000,6,60.000000,0,0.000000,0,0.000000,3,30.000000,1,10.000000,0,0.000000
6,Bremen,5,0,0.000000,1,20.000000,0,0.000000,0,0.000000,4,80.000000,0,0.000000,0,0.000000
7,Hamburg,10,0,0.000000,7,70.000000,0,0.000000,0,0.000000,2,20.000000,1,10.000000,0,0.000000
8,Hessen,22,0,0.000000,7,31.818182,0,0.000000,1,4.545455,14,63.636364,0,0.000000,0,0.000000
9,Nordrhein-Westfalen,42,0,0.000000,22,52.380952,0,0.000000,0,0.000000,17,40.476190,2,4.761905,1,2.380952


,Region,Global #,Without SSL #,Without SSL %,2048 (RSA) #,2048 (RSA) %,3072 (RSA) #,3072 (RSA) %,256 (ECC) #,256 (ECC) %,4096 (RSA) #,4096 (RSA) %,384 (ECC) #,384 (ECC) %,8192 (RSA) #,8192 (RSA) %
0,Baden-Württemberg,24,0,0.0,20,83.333333,0,0.000000,0,0.000000,3,12.500000,1,4.166667,0,0.0
1,Bayern,12,0,0.0,9,75.000000,1,8.333333,1,8.333333,1,8.333333,0,0.000000,0,0.0
2,Berlin,28,0,0.0,23,82.142857,0,0.000000,1,3.571429,4,14.285714,0,0.000000,0,0.0
3,Brandenburg,9,0,0.0,8,88.888889,0,0.000000,0,0.000000,1,11.111111,0,0.000000,0,0.0
4,Bremen,2,0,0.0,1,50.000000,0,0.000000,0,0.000000,1,50.000000,0,0.000000,0,0.0
5,Hamburg,11,0,0.0,9,81.818182,0,0.000000,1,9.090909,1,9.090909,0,0.000000,0,0.0
6,Hessen,23,0,0.0,18,78.260870,1,4.347826,0,0.000000,4,17.391304,0,0.000000,0,0.0
7,Niedersachsen,16,0,0.0,14,87.500000,0,0.000000,0,0.000000,2,12.500000,0,0.000000,0,0.0
8,Nordrhein-Westfalen,25,0,0.0,18,72.000000,0,0.000000,0,0.000000,7,28.000000,0,0.000000,0,0.0
9,Rheinland-Pfalz,6,0,0.0,6,100.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0


In [ ]:
# Analyze of SSL Algorithms by region
print(source_df['key_alg'].unique())
# settings
column_to_sort = 'Without SSL (Public) %'
sort_ascending = False
table_name = 'SSL_Algorithms_by_region'

columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without SSL by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Public)', criteria=f'grade == "M" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Private)', criteria=f'grade == "M" & {only_private}', columns_to_display=columns_to_display)
# Creating column with the distribution of SSL Algorithms by region
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='RSA (Public)', criteria=f'grade != "M" & key_alg == "RSA" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='RSA (Private)', criteria=f'grade != "M" & key_alg == "RSA" & {only_private}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='ECC (Public)', criteria=f'grade != "M" & key_alg == "EC" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='ECC (Private)', criteria=f'grade != "M" & key_alg == "EC" & {only_private}', columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

In [ ]:
# Analyze of SSL Algorithms by region (Pub/Pvt)
# settings
column_to_sort = 'Without SSL %'
sort_ascending = False
config = [
    {'table_name': 'SSL_Algorithms_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'SSL_Algorithms_by_region_private', 'hei_type': 'Private'}
]
dfs = []
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)
    # create columns
    # Column creation with distribution of records without SSL by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL', criteria=f'grade == "M" & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Creating column with the distribution of SSL Algorithms by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='RSA', criteria=f'grade != "M" & key_alg == "RSA" & category == "{hei_type}"', columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='ECC', criteria=f'grade != "M" & key_alg == "EC" & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)

df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')


ranks_columns = []
# Specify the columns to use
columns_to_use = ['ECC', 'RSA', 'Without SSL']
# Apply the function to calculate the rank
for i, column in enumerate(columns_to_use):
    rank_colum = f'Rank {i}'
    ranks_columns.append(rank_colum)
    df_combined[rank_colum] = df_combined.apply(lambda row: calculate_rank(row, column), axis=1)

#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=ranks_columns, ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=ranks_columns, inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]


save_table(df_combined, category=category, table_name='SSL_Algorithms_by_region_combined')

In [ ]:
# Report in latex
report_results = get_extreme_values(analysis_df)
report_name = 'SSL_Algorithms_by_region'

tot_pub = report_results.get("Total").get("RSA (Public) #") + report_results.get("Total").get("ECC (Public) #") + report_results.get("Total").get("Without SSL (Public) #")
tot_priv = report_results.get("Total").get("RSA (Private) #") + report_results.get("Total").get("ECC (Private) #") + report_results.get("Total").get("Without SSL (Private) #")
hei_public_rsa = format(report_results.get("Total").get("RSA (Public) #") / tot_pub * 100, ".2f")
hei_public_ecc = format(report_results.get("Total").get("ECC (Public) #") / tot_priv * 100, ".2f")

hei_private_rsa = format(report_results.get("Total").get("RSA (Private) %"), ".2f")
hei_private_ecc = format(report_results.get("Total").get("ECC (Private) %"), ".2f")

report_figure = f"""
\\begin{{figure}}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{{charts/{report_name}.pdf}}
    \caption{{Distribution of the type of \gls{{ssl}}/\gls{{tls}} algorithms used}}\label{{fig:ssl-algorithms}}
\end{{figure}}
"""

report = f'{report_figure}\n\n'
report += f"""
Fig.~\\ref{{fig:ssl-algorithms}} presents an overview of the use of the type of \gls{{ssl}}/\gls{{tls}} at \glspl{{hei}} in \countryName.

According to the data, {hei_public_rsa}\% of the public institutions analyzed, and {hei_private_rsa}\% of the private institutions are using \gls{{rsa}} encryption algorithm for \gls{{ssl}}/\gls{{tls}}.

On the other hand, {hei_public_ecc}\% of the public institutions analyzed, and {hei_private_ecc}\% of the private institutions are using \gls{{ecc}} encryption algorithm for \gls{{ssl}}/\gls{{tls}}.

In terms of regional differences, private institutions in {report_results.get("RSA (Private) %").get("top_regions")[0][0]} ({format(report_results.get("RSA (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("RSA (Private) %").get("top_regions")[1][0]} ({format(report_results.get("RSA (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("RSA (Private) %").get("top_regions")[2][0]} ({format(report_results.get("RSA (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("RSA (Public) %").get("top_regions")[0][0]} ({format(report_results.get("RSA (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("RSA (Public) %").get("top_regions")[1][0]} ({format(report_results.get("RSA (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("RSA (Public) %").get("top_regions")[2][0]} ({format(report_results.get("RSA (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of \gls{{rsa}} encryption algorithm on your websites.

In contrast, private institutions in, {report_results.get("RSA (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("RSA (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("RSA (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("RSA (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("RSA (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("RSA (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("RSA (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("RSA (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("RSA (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("RSA (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("RSA (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("RSA (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of \gls{{rsa}} encryption algorithm on your websites.

Finally, private institutions in {report_results.get("ECC (Private) %").get("top_regions")[0][0]} ({format(report_results.get("ECC (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("ECC (Private) %").get("top_regions")[1][0]} ({format(report_results.get("ECC (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("ECC (Private) %").get("top_regions")[2][0]} ({format(report_results.get("ECC (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("ECC (Public) %").get("top_regions")[0][0]} ({format(report_results.get("ECC (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("ECC (Public) %").get("top_regions")[1][0]} ({format(report_results.get("ECC (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("ECC (Public) %").get("top_regions")[2][0]} ({format(report_results.get("ECC (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of \gls{{ecc}} encryption algorithm on your websites.

In contrast, private institutions in {report_results.get("ECC (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("ECC (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("ECC (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("ECC (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("ECC (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("ECC (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("ECC (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("ECC (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("ECC (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("ECC (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("ECC (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("ECC (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of \gls{{ecc}} encryption algorithm on your websites.
"""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)

In [ ]:
# Analyze of Worst supported SSL/TLS versions by region (Pub/Pvt)

# settings
column_to_sort = 'Without SSL %'
sort_ascending = False
config = [
    {'table_name': 'Worst_SSL_supported_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'Worst_SSL_supported_by_region_private', 'hei_type': 'Private'}
]
dfs = []
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)
    # create columns
    # Column creation with distribution of records without SSL by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL', criteria=f'grade == "M" & category == "{hei_type}"', columns_to_display=columns_to_display)

    # Creating column with the distribution of Worst supported SSL/TLS versions by region
    versions = ['SSLv2.0', 'SSLv3.0', 'TLSv1.0', 'TLSv1.1', 'TLSv1.2', 'TLSv1.3']
    only_https = 'grade != "M" &'
    for i in range(len(versions)):
        current_version = f'`{versions[i]}` == True &' if i != 0 else f'`{versions[i]}` == True'
        previous_versions = ' & '.join([f'`{versions[j]}` == False' for j in range(i)])
        criteria = f'{only_https} {current_version} {previous_versions}'
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f'{versions[i]}', criteria=f'{criteria} & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)


df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')


ranks_columns = []
# Specify the columns to use
columns_to_use = ['TLSv1.3', 'TLSv1.2']
# Apply the function to calculate the rank
for i, column in enumerate(columns_to_use):
    rank_colum = f'Rank {i}'
    ranks_columns.append(rank_colum)
    df_combined[rank_colum] = df_combined.apply(lambda row: calculate_rank(row, column), axis=1)

#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=ranks_columns, ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=ranks_columns, inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]


save_table(df_combined, category=category, table_name='Worst_SSL_supported_by_region_combined')

In [ ]:
# Report in latex
report_results = get_extreme_values(analysis_df)
report_name = 'Worst_SSL_supported_by_region'

hei_public_ssl2 = format(report_results.get("Total").get("SSLv2.0 (Public) %"), ".2f")
hei_public_ssl3 = format(report_results.get("Total").get("SSLv3.0 (Public) %"), ".2f")
hei_public_tls10 = format(report_results.get("Total").get("TLSv1.0 (Public) %"), ".2f")
hei_public_tls11 = format(report_results.get("Total").get("TLSv1.1 (Public) %"), ".2f")
hei_public_tls12 = format(report_results.get("Total").get("TLSv1.2 (Public) %"), ".2f")
hei_public_tls13 = format(report_results.get("Total").get("TLSv1.3 (Public) %"), ".2f")

hei_private_ssl2 = format(report_results.get("Total").get("SSLv2.0 (Private) %"), ".2f")
hei_private_ssl3 = format(report_results.get("Total").get("SSLv3.0 (Private) %"), ".2f")
hei_private_tls10 = format(report_results.get("Total").get("TLSv1.0 (Private) %"), ".2f")
hei_private_tls11 = format(report_results.get("Total").get("TLSv1.1 (Private) %"), ".2f")
hei_private_tls12 = format(report_results.get("Total").get("TLSv1.2 (Private) %"), ".2f")
hei_private_tls13 = format(report_results.get("Total").get("TLSv1.3 (Private) %"), ".2f")

report_figure = f"""
\\begin{{figure}}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{{charts/{report_name}.pdf}}
    \caption{{Distribution of the worst version of \gls{{ssl}}/\gls{{tls}} protocols.}}\label{{fig:ssl-worst}}
\end{{figure}}
"""

report = f'{report_figure}\n\n'
report += f"""
Fig.~\\ref{{fig:ssl-worst}} presents an overview of the worst version of \gls{{ssl}}/\gls{{tls}} protocols at \glspl{{hei}} in \countryName.

According to the data, {hei_public_ssl2}\% of the public institutions analyzed, and {hei_private_ssl2}\% of the private institutions are using SSLv2.0 version protocol, {hei_public_ssl3}\% of the public institutions analyzed, and {hei_private_ssl3}\% of the private institutions are using SSLv3.0 version protocol, {hei_public_tls10}\% of the public institutions analyzed, and {hei_private_tls10}\% of the private institutions are using TLSv1.0 version protocol, {hei_public_tls11}\% of the public institutions analyzed, and {hei_private_tls11}\% of the private institutions are using TLSv1.1 version protocol, {hei_public_tls12}\% of the public institutions analyzed, and {hei_private_tls12}\% of the private institutions are using TLSv1.2 version protocol, and {hei_public_tls13}\% of the public institutions analyzed, and {hei_private_tls13}\% of the private institutions are using TLSv1.3 version protocol.

In terms of regional differences, private institutions in {report_results.get("SSLv2.0 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("SSLv2.0 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("SSLv2.0 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("SSLv2.0 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv2.0 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("SSLv2.0 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("SSLv2.0 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("SSLv2.0 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("SSLv2.0 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("SSLv2.0 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv2.0 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("SSLv2.0 (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of SSLv2.0 version protocol on your websites.

In contrast, private institutions in {report_results.get("SSLv2.0 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("SSLv2.0 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("SSLv2.0 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("SSLv2.0 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv2.0 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("SSLv2.0 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("SSLv2.0 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("SSLv2.0 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("SSLv2.0 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("SSLv2.0 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv2.0 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("SSLv2.0 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of SSLv2.0 version protocol on your websites.

While the usage of SSLv3.0 version protocol is higher in private institutions in {report_results.get("SSLv3.0 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("SSLv3.0 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("SSLv3.0 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("SSLv3.0 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv3.0 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("SSLv3.0 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("SSLv3.0 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("SSLv3.0 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("SSLv3.0 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("SSLv3.0 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv3.0 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("SSLv3.0 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, the usage of SSLv3.0 version protocol is lower in private institutions in {report_results.get("SSLv3.0 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("SSLv3.0 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("SSLv3.0 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("SSLv3.0 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv3.0 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("SSLv3.0 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("SSLv3.0 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("SSLv3.0 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("SSLv3.0 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("SSLv3.0 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("SSLv3.0 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("SSLv3.0 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

The usage of TLSv1.0 version protocol is higher in private institutions in {report_results.get("TLSv1.0 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.0 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.0 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.0 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.0 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.0 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.0 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.0 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.0 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.0 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.0 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.0 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.0 version protocol is lower in private institutions in {report_results.get("TLSv1.0 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.0 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.0 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.0 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.0 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.0 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.0 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.0 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.0 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.0 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.0 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.0 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

Already the use of TLSv1.1 version protocol is higher in private institutions in {report_results.get("TLSv1.1 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.1 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.1 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.1 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.1 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.1 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.1 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.1 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.1 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.1 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.1 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.1 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.1 version protocol is lower in private institutions in {report_results.get("TLSv1.1 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.1 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.1 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.1 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.1 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.1 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.1 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.1 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.1 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.1 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.1 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.1 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

While the use of TLSv1.2 version protocol is higher in private institutions in {report_results.get("TLSv1.2 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.2 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.2 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.2 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.2 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.2 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.2 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.2 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.2 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.2 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.2 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.2 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.2 version protocol is lower in private institutions in {report_results.get("TLSv1.2 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.2 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.2 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.2 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.2 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.2 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.2 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.2 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.2 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.2 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.2 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.2 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.

Finally, the use of TLSv1.3 version protocol is higher in private institutions in {report_results.get("TLSv1.3 (Private) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.3 (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.3 (Private) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.3 (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.3 (Private) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.3 (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.3 (Public) %").get("top_regions")[0][0]} ({format(report_results.get("TLSv1.3 (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.3 (Public) %").get("top_regions")[1][0]} ({format(report_results.get("TLSv1.3 (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.3 (Public) %").get("top_regions")[2][0]} ({format(report_results.get("TLSv1.3 (Public) %").get("top_regions")[2][1], ".2f")}\%) than in other regions, while the usage of TLSv1.3 version protocol is lower in private institutions in {report_results.get("TLSv1.3 (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.3 (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.3 (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.3 (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.3 (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.3 (Private) %").get("bottom_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("TLSv1.3 (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("TLSv1.3 (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("TLSv1.3 (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("TLSv1.3 (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("TLSv1.3 (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("TLSv1.3 (Public) %").get("bottom_regions")[2][1], ".2f")}\%) than in other regions.
"""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)


In [ ]:
# Analyze of valid SSL/TLS by region (Pub/Pvt)

# settings
column_to_sort = 'Without SSL %'
sort_ascending = False
config = [
    {'table_name': 'valid_ssl_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'valid_ssl_by_region_private', 'hei_type': 'Private'}
]
dfs = []
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)
    # create columns
    # Column creation with distribution of records without SSL by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL', criteria=f'grade == "M" & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Creating column with the distribution of valid SSL/TLS by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Valid Configuration', criteria=f'grade != "M" & is_valid == True & category == "{hei_type}"', columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Invalid Configuration', criteria=f'grade != "M" & is_valid == False & category == "{hei_type}"', columns_to_display=columns_to_display)
    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)


df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')


ranks_columns = []
# Specify the columns to use
columns_to_use = ['Valid Configuration', 'Invalid Configuration', 'Without SSL']
# Apply the function to calculate the rank
for i, column in enumerate(columns_to_use):
    rank_colum = f'Rank {i}'
    ranks_columns.append(rank_colum)
    df_combined[rank_colum] = df_combined.apply(lambda row: calculate_rank(row, column), axis=1)

#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=ranks_columns, ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=ranks_columns, inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]


save_table(df_combined, category=category, table_name='valid_ssl_by_region_combined')

In [ ]:
# Analyze of valid SSL/TLS by region

# settings
column_to_sort = 'Without SSL (Public) %'
sort_ascending = False
table_name = 'valid_ssl_by_region'

columns_to_display = [region_column_name.title(), column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without SSL by region
only_public = 'category == "Public"'
only_private = 'category == "Private"'
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Public)', criteria=f'grade == "M" & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without SSL (Private)', criteria=f'grade == "M" & {only_private}', columns_to_display=columns_to_display)
# Creating column with the distribution of valid SSL/TLS by region
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Valid Configuration (Public)', criteria=f'grade != "M" & is_valid == True & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Invalid Configuration (Public)', criteria=f'grade != "M" & is_valid == False & {only_public}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Valid Configuration (Private)', criteria=f'grade != "M" & is_valid == True & {only_private}', columns_to_display=columns_to_display)
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Invalid Configuration (Private)', criteria=f'grade != "M" & is_valid == False & {only_private}', columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)

In [ ]:
# Report in latex
report_results = get_extreme_values(analysis_df)
report_name = 'valid_ssl_by_region'

tot_pub = report_results.get("Total").get("Valid Configuration (Public) #") + report_results.get("Total").get("Invalid Configuration (Public) #") + report_results.get("Total").get("Without SSL (Public) #")
hei_public_valid = format(report_results.get("Total").get("Valid Configuration (Public) #") / tot_pub * 100, ".2f")

hei_public_invalid = format(report_results.get("Total").get("Invalid Configuration (Public) %"), ".2f")
tot_private = report_results.get("Total").get("Valid Configuration (Private) #") + report_results.get("Total").get("Invalid Configuration (Private) #") + report_results.get("Total").get("Without SSL (Private) #")
hei_private_valid = format(report_results.get("Total").get("Valid Configuration (Private) #") / tot_private * 100, ".2f")
hei_private_invalid = format(report_results.get("Total").get("Invalid Configuration (Private) %"), ".2f")

report_figure = f"""
\\begin{{figure}}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{{charts/{report_name}.pdf}}
    \caption{{Distribution of valid configuration \gls{{ssl}}/\gls{{tls}} by region.}}\label{{fig:valid-ssl}}
\end{{figure}}
"""

report = f'{report_figure}\n\n'

report += f"""
Fig.~\\ref{{fig:valid-ssl}} presents an overview of the valid configuration of \gls{{ssl}}/\gls{{tls}} protocols at \glspl{{hei}} in \countryName.

According to the data, \gls{{ssl}}/\gls{{tls}} protocols are configured correctly in {hei_public_valid}\% of the \glspl{{hei}} public, and in {hei_private_valid}\% of the \glspl{{hei}} private.


In terms of regional differences, private institutions in {report_results.get("Valid Configuration (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Valid Configuration (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Valid Configuration (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Valid Configuration (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Valid Configuration (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Valid Configuration (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Valid Configuration (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Valid Configuration (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Valid Configuration (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Valid Configuration (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Valid Configuration (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Valid Configuration (Public) %").get("top_regions")[2][1], ".2f")}\%) have the highest percentage of valid configuration of \gls{{ssl}}/\gls{{tls}} protocols.

In contrast, private institutions in {report_results.get("Invalid Configuration (Private) %").get("top_regions")[0][0]} ({format(report_results.get("Invalid Configuration (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Invalid Configuration (Private) %").get("top_regions")[1][0]} ({format(report_results.get("Invalid Configuration (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Invalid Configuration (Private) %").get("top_regions")[2][0]} ({format(report_results.get("Invalid Configuration (Private) %").get("top_regions")[2][1], ".2f")}\%), and public institutions in {report_results.get("Invalid Configuration (Public) %").get("top_regions")[0][0]} ({format(report_results.get("Invalid Configuration (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("Invalid Configuration (Public) %").get("top_regions")[1][0]} ({format(report_results.get("Invalid Configuration (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("Invalid Configuration (Public) %").get("top_regions")[2][0]} ({format(report_results.get("Invalid Configuration (Public) %").get("top_regions")[2][1], ".2f")}\%) have the highest percentage of invalid configuration of \gls{{ssl}}/\gls{{tls}} protocols.
"""

print(report)
# save report to file txt
save_report(report=report, category=category, report_name=report_name)